In [1]:
import pandas as pd
from datetime import date

In [2]:
df_menu = pd.read_csv("dataset/menu.csv")
df_order = pd.read_csv("dataset/order.csv")
df_promotion = pd.read_csv("dataset/promotion.csv")

In [3]:
df_order

,order_id,menu_id,quantity,sales_date
0,1,3,5,2021/01/01
1,1,3,5,2021/01/01
2,1,4,3,2021/01/01
3,1,4,4,2021/01/01
4,1,3,5,2021/01/01
...,...,...,...,...
994,298,3,4,2021/05/01
995,298,3,3,2021/05/01
996,299,4,1,2021/05/01
997,299,4,2,2021/05/01


In [4]:
df_promotion['start_date'] = pd.to_datetime(df_promotion['start_date']).dt.date
df_promotion['end_date'] = pd.to_datetime(df_promotion['end_date']).dt.date
df_order['sales_date'] = pd.to_datetime(df_order['sales_date']).dt.date
df_order['order_id'] = df_order['order_id'].astype('string')


In [5]:
def convert_order_id(id,alias):
    return id+alias

df_order['order_id'] = df_order['order_id'].apply(lambda x: convert_order_id(x,'_csv'))

In [6]:
def add_promotion_id(sales_dt):
    promotion = df_promotion[(sales_dt >= df_promotion['start_date']) & (sales_dt <= df_promotion['end_date'] ) ][['id']]
    if promotion.shape[0] == 0:
        promotion_id = -1
    else:
        promotion_id = promotion.iloc[0,0]
    return promotion_id
df_order['promotion_id'] =  df_order['sales_date'].apply(add_promotion_id)

In [7]:
df_menu_agg = df_menu.groupby('menu_id').agg({'effective_date':'max'}).reset_index()

In [8]:
df_menu_agg

,menu_id,effective_date
0,1,2021-03-09
1,2,2021-01-01
2,3,2021-02-01
3,4,2021-04-01


In [9]:
df_menu = df_menu.merge(df_menu_agg,on=['menu_id','effective_date'])

In [10]:
df_result  = pd.merge(df_order,df_promotion,left_on='promotion_id',right_on='id',how='left') \
            .merge(df_menu,on='menu_id')

In [11]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        999 non-null    object 
 1   menu_id         999 non-null    int64  
 2   quantity        999 non-null    int64  
 3   sales_date      999 non-null    object 
 4   promotion_id    999 non-null    int64  
 5   id              125 non-null    float64
 6   start_date      125 non-null    object 
 7   end_date        125 non-null    object 
 8   disc_value      125 non-null    float64
 9   max_disc        125 non-null    float64
 10  brand           999 non-null    object 
 11  name            999 non-null    object 
 12  price           999 non-null    int64  
 13  cogs            999 non-null    int64  
 14  effective_date  999 non-null    object 
dtypes: float64(3), int64(5), object(7)
memory usage: 117.2+ KB


In [12]:
df_result[['order_id','menu_id','promotion_id','id']] = df_result[['order_id','menu_id','promotion_id','id']].astype('object')
df_result[['brand','name']] = df_result[['brand','name']].astype('string')
df_result[['quantity']] = df_result[['quantity']].astype('int')
df_result[['disc_value','max_disc','price','cogs']] = df_result[['disc_value','max_disc','price','cogs']].astype('float')
df_result['sales_date'] = pd.to_datetime(df_result['sales_date'])
df_result['start_date'] = pd.to_datetime(df_result['start_date'])
df_result['end_date'] = pd.to_datetime(df_result['end_date'])
df_result['effective_date'] = pd.to_datetime(df_result['effective_date'])

In [13]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        999 non-null    object        
 1   menu_id         999 non-null    object        
 2   quantity        999 non-null    int64         
 3   sales_date      999 non-null    datetime64[ns]
 4   promotion_id    999 non-null    object        
 5   id              125 non-null    object        
 6   start_date      125 non-null    datetime64[ns]
 7   end_date        125 non-null    datetime64[ns]
 8   disc_value      125 non-null    float64       
 9   max_disc        125 non-null    float64       
 10  brand           999 non-null    string        
 11  name            999 non-null    string        
 12  price           999 non-null    float64       
 13  cogs            999 non-null    float64       
 14  effective_date  999 non-null    datetime64[ns]
dtypes: dat

In [14]:
df_result.loc[df_result['disc_value'] > 1,'disc_value'] = 1

In [15]:
df_result.head(2)

,order_id,menu_id,quantity,sales_date,promotion_id,id,start_date,end_date,disc_value,max_disc,brand,name,price,cogs,effective_date
0,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
1,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01


In [16]:
today = date(2021,1,1)
df_result = df_result[df_result['sales_date'].dt.date >= today]

In [17]:
df_result.head(2)

,order_id,menu_id,quantity,sales_date,promotion_id,id,start_date,end_date,disc_value,max_disc,brand,name,price,cogs,effective_date
0,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
1,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01


In [18]:
df_filter_sales_date_based_order_id =  df_result.groupby(['order_id']).agg({'sales_date':'min'}).reset_index()

In [19]:
def convert_sales_date(order_id,df):
    return  df[df['order_id'] == order_id]['sales_date'].iloc[0]

df_result['sales_date'] =  df_result.apply(lambda row: convert_sales_date(row.order_id,df_filter_sales_date_based_order_id),axis=1)
# df_result['sales_date'] = pd.to_datetime(df_result['sales_date'])

In [20]:
df_result['start_date'].duplicated()

0      False
1       True
2       True
3       True
4       True
       ...  
994     True
995     True
996     True
997     True
998     True
Name: start_date, Length: 999, dtype: bool

In [21]:
# df_result.drop_duplicates(inplace=True)

In [24]:
df_result[df_result.duplicated(subset=['order_id','menu_id','sales_date'],keep=False)]

,order_id,menu_id,quantity,sales_date,promotion_id,id,start_date,end_date,disc_value,max_disc,brand,name,price,cogs,effective_date
0,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
1,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
2,1_csv,4,3,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Z,Burger,75000.0,22000.0,2021-04-01
3,1_csv,4,4,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Z,Burger,75000.0,22000.0,2021-04-01
4,1_csv,3,5,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,297_csv,3,5,2021-04-30,-1,NaN,NaT,NaT,NaN,NaN,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
994,298_csv,3,4,2021-05-01,-1,NaN,NaT,NaT,NaN,NaN,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
995,298_csv,3,3,2021-05-01,-1,NaN,NaT,NaT,NaN,NaN,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01
996,299_csv,4,1,2021-05-01,-1,NaN,NaT,NaT,NaN,NaN,Z,Burger,75000.0,22000.0,2021-04-01


In [41]:
column = df_result.columns.to_list()

In [43]:
column.remove("quantity")

In [48]:
df_result_group = df_result.groupby(column).agg({'quantity':'sum'}).reset_index()

In [52]:
df_result_group[df_result_group['order_id'] == '1_csv']

,order_id,menu_id,sales_date,promotion_id,id,start_date,end_date,disc_value,max_disc,brand,name,price,cogs,effective_date,quantity
29,1_csv,3,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Y,10pcs Chicken Wings,110000.0,42000.0,2021-02-01,15
30,1_csv,4,2021-01-01,1,1.0,2021-01-01,2021-01-03,0.2,40000.0,Z,Burger,75000.0,22000.0,2021-04-01,7
